<a href="https://colab.research.google.com/github/amoyag/ExData_Plotting1/blob/master/nb_caffeine_mod_for_AFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *
import sys
import matplotlib
init()

In [ ]:
from google.colab import drive
import os

drive.flush_and_unmount()

drive.mount('/content/drive')

path = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb'
path_aft = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aft.sdf'
path_params = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/AFT.params'
drive_path = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs'


Mounted at /content/drive


In [ ]:
pose = pose_from_pdb(path)



core.import_pose.import_pose: File '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 12 atoms at position 121 in file /content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb. Best match rsd_type:  pdb_AFT
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue VAL:CtermProteinFull 238


Este código no tiene que ir aquí. Estaba en el cuaderno jupyter que te indiqué que debías consultar. Solo te sirve para que veas cómo se hace la estructura inicial del complejo Nb-AFL. Que ya he prepaarado yo. Te expliqué que de ese cuaderno jupyter tienes que implementar el código para hacer el High resolution Local Docking. Toda la parte de ligand y target preparation ya la hice yo.

In [ ]:
!apt-get update
!apt-get install -y libarchive-dev
from google.colab import files
uploaded = files.upload()
!apt-get install -y openbabel

# Descargar Miniconda para Linux
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh

# Descargar Miniconda para macOS
# !curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-x86_64.pkg

# Descargar Miniconda para Windows
# !curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe

# Hacer el script ejecutable para Linux
!chmod +x miniconda.sh

# Ejecutar el instalador de Miniconda para Linux
!./miniconda.sh -b -p $HOME/miniconda3

# Agregar Miniconda al PATH en Linux
import os
os.environ["PATH"] += os.pathsep + os.path.join(os.getenv('HOME'), 'miniconda3', 'bin')

# Actualizar Miniconda
!conda update conda -y

# Instalar Open Babel
!conda install -c conda-forge openbabel -y

# Instalar Jupyter Notebook
!conda install -c conda-forge notebook -y

# Verificar `conda`
!conda --version

# Verificar `obabel`
!obabel -V

# Verificar Jupyter Notebook
!jupyter --version

# Ejecutar Jupyter Notebook
# !jupyter notebook

!conda install -c conda-forge openbabel --yes
!conda config --set solver classic

!obabel aft.sdf -O aft_conformers.sdf --confab --conf 10 --original --verbose --ecutoff 5
!obabel aft_conformers.sdf -O aft_conformers.pdb



In [ ]:
ligand_parameters = path_params
flags = f"""
-extra_res_fa {ligand_parameters}
-s path
-ex1
-ex2
-no_optH false
-ignore_ligand_chi true
-overwrite
-restore_pre_talaris_2013_behavior true
"""
pyrosetta.distributed.init(flags)

In [ ]:
pose = pyrosetta.io.pose_from_file(filename = path)

core.import_pose.import_pose: File '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 12 atoms at position 121 in file /content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb. Best match rsd_type:  pdb_AFT
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue VAL:CtermProteinFull 238


In [ ]:
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>

		<SCOREFXNS>
			<ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep">
			</ScoreFunction>
			<ScoreFunction name="hard_rep" weights="ligand">
			</ScoreFunction>
		</SCOREFXNS>

		<LIGAND_AREAS>
			<LigandArea name="inhibitor_dock_sc" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
			<LigandArea name="inhibitor_final_sc" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
			<LigandArea name="inhibitor_final_bb" chain="X" cutoff="7.0" add_nbr_radius="false" all_atom_mode="true" Calpha_restraints="0.3"/>
		</LIGAND_AREAS>

		<INTERFACE_BUILDERS>
			<InterfaceBuilder name="side_chain_for_docking" ligand_areas="inhibitor_dock_sc"/>
			<InterfaceBuilder name="side_chain_for_final" ligand_areas="inhibitor_final_sc"/>
			<InterfaceBuilder name="backbone" ligand_areas="inhibitor_final_bb" extension_window="3"/>
		</INTERFACE_BUILDERS>

		<MOVEMAP_BUILDERS>
			<MoveMapBuilder name="docking" sc_interface="side_chain_for_docking" minimize_water="false"/>
			<MoveMapBuilder name="final" sc_interface="side_chain_for_final" bb_interface="backbone" minimize_water="false"/>
		</MOVEMAP_BUILDERS>

		<SCORINGGRIDS ligand_chain="X" width="15">
			<ClassicGrid grid_name="classic" weight="1.0"/>
		</SCORINGGRIDS>

		<MOVERS>
			<Transform name="transform" chain="X" box_size="7.0" move_distance="0.2" angle="20" cycles="30" repeats="1" temperature="5"/>
			<HighResDocker name="high_res_docker" cycles="6" repack_every_Nth="3" scorefxn="ligand_soft_rep" movemap_builder="docking"/>
			<FinalMinimizer name="final" scorefxn="hard_rep" movemap_builder="final"/>
			<InterfaceScoreCalculator name="add_scores" chains="X" scorefxn="hard_rep" native="aftNb_start.pdb"/>
		</MOVERS>

		<PROTOCOLS>
			<Add mover_name="transform"/>
			<Add mover_name="high_res_docker"/>
			<Add mover_name="final"/>
			<Add mover_name="add_scores"/>
		</PROTOCOLS>


</ROSETTASCRIPTS>
""").get_mover("ParsedProtocol")

protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: ...done
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: ...done
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep"/>
		<ScoreFunction name="hard_rep" weights="ligand"/>
	</SCOREFXNS>
	<LIGAND_AREAS>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="X" cutoff="6.0" name="inhibitor_dock_sc"/>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="X" cutoff="6.0" name="inhibitor_final_sc"/>
		<LigandArea Calpha_restraints="0.3" add_nbr_radius="false" all_atom_mode="true" chain

RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/import_pose/import_pose.cc:361
[ ERROR ] UtilityExitException
ERROR: Cannot open file "aftNb_start.pdb"

